# Lecture 6.3: Multiple Regressions II

## Outline

* Dummy variables for indicator and categorical data
* Interaction terms
    * Interactions between continuous and categorical variables
    * Interactions between continuous and continuous variables
* Variable selection - stepwise regression

In [ ]:
%pylab inline
import pandas as pd
import statsmodels.api as sm

from seaborn import lmplot

In [ ]:
fake_puppy_data = pd.DataFrame()
fake_puppy_data['num_puppies'] = np.abs(np.random.normal(3,4,size=1000)).round()
fake_puppy_data[fake_puppy_data['num_puppies']==0] = 1
other_animals = ['yes','no']
fake_puppy_data['has_other_animals'] = np.random.choice(other_animals,replace=True,size=1000,p=[0.2,0.8])
age_groups = ['puppy','dog','elderly dog']
fake_puppy_data['age_group'] = np.random.choice(age_groups,replace=True,size=1000,p=[0.5,0.2,0.3])
fake_puppy_data['score'] = 4.231  * fake_puppy_data['num_puppies'] + \
                           24.213 * (fake_puppy_data['has_other_animals']=='yes') + \
                           -0.340 * (fake_puppy_data['has_other_animals']=='no') + \
                           20.453 * (fake_puppy_data['age_group']=='puppy') + \
                           -6.543 * (fake_puppy_data['age_group']=='dog') + \
                           14.568 * (fake_puppy_data['age_group']=='elderly dog') + \
                           6.574  * (fake_puppy_data['num_puppies'] * 1.*(fake_puppy_data['has_other_animals'] == 'yes')) + \
                           3.486  * (fake_puppy_data['num_puppies'] * 1.*(fake_puppy_data['age_group'] == 'puppy')) + \
                           0.182  * (fake_puppy_data['num_puppies'] * 1.*(fake_puppy_data['age_group'] == 'elderly dog')) + \
                           12.381 + \
                           np.random.normal(15,3,size=1000)
fake_puppy_data['score'] = 100 * ((fake_puppy_data['score']-fake_puppy_data['score'].min())/(fake_puppy_data['score'].max() - fake_puppy_data['score'].min()))

## More stories

My wife likes pictures of puppies. To document this I had her look at and score 1000 photos of puppies (I did not actually do this one in real life).

In [ ]:
fake_puppy_data.head(15)

### Data dictionary

* num_puppies is number of puppies in the photo
* has_other_animals is yes if there are other animals in photo, no otherwise
* age_group describes the average age of the dogs in the photo, either puppy, dog, or elderly dog



### Question:
Can I run my current data through statsmodels?

In [ ]:
X = fake_puppy_data[['num_puppies','has_other_animals','age_group']]
y = fake_puppy_data.score
model = sm.OLS(y, X)

## Any ideas on how to deal with this?

## Add dummy variables

In [ ]:
data = fake_puppy_data.copy()

In [ ]:
data['others_yes'] = 1 * (data.has_other_animals == 'yes') 
data.head(15)

## How the dummy variable works  
  

$x_i = \left\{\begin{array}{11} 1 & \mbox{if ith picture has other animals} \\ 0 & \mbox{if ith picture has no other animals} \end{array} \right. $

$$y_i \sim \beta_0 + \beta_1 x_i + \epsilon_i = \left\{\begin{array}{11} \beta_0 + \beta_1 + \epsilon_i & \mbox{if ith picture has other animals} \\ \beta_0 + \epsilon_i & \mbox{if ith picture has no other animals} \end{array} \right.$$

## Important 

When you do this you are changing the interpretation of your intercept.  
* What did the intercept mean before?
* What does it mean now?

## Categorical variables

* Basically the same as dummy variables, but with more than two levels
* In general you need number of categories - 1 columns to encode a categorical variable

In [ ]:
data['puppy'] = 1 * (data.age_group == 'puppy')
data['elderly'] = 1 * (data.age_group == 'elderly dog')
data.head(15)

## How this works  
  

$x_{i1} = \left\{\begin{array}{11} 1 & \mbox{if the ith picture has mostly puppies} \\ 0 & \mbox{if the ith picture does not have mostly puppies} \end{array} \right. $

$x_{i2} = \left\{\begin{array}{11} 1 & \mbox{if the ith picture has mostly elderly dogs} \\ 0 & \mbox{if the ith picture does not have mostly elderly dogs} \end{array} \right. $

$$y_i \sim \beta_0 + \beta_1 x_{i1} + \beta_2 x_{i2} + \epsilon_i = \left\{\begin{array}{111} \beta_0 + \beta_1 + \epsilon_i & \mbox{if ith picture has mostly puppies} \\ \beta_0 + \beta_2 + \epsilon_i & \mbox{if ith picture has mostly elderly dogs} \\ \beta_0 + \epsilon_i & \mbox{if ith picture has mostly middle aged dogs}\end{array} \right.$$

## Once again
Interpretation of coefficients changes

* What is $\beta_0$ now?
* What is $\beta_1$?
* What is $\beta_2$?

## How do we use this?

In [ ]:
data.head()

In [ ]:
y = data.score
X = data[['num_puppies','others_yes','puppy','elderly']]
X = sm.add_constant(X)  # <- add in constant term

model = sm.OLS(y, X)
results = model.fit()
results.summary()

## Note before we move on

You can add dummy variables to a data frame automatically with pd.get_dummies

### Important
This adds columns for all categories, so it does not have a baseline, you can remove the columns you don't want afterwards

In [ ]:
test = pd.get_dummies(fake_puppy_data)
test.head()

Our model before was fairly good, but how to investigate?

Since we only have 1 continuous variable we can visualize in two dimensions, and we would expect different offsets for each line, but the same slope.

In [ ]:
data.plot(x='num_puppies', y='score', kind='scatter');

## Lets try to clear this up a little

In [ ]:
p_data = data[data.puppy == 1]
p_data.plot(x='num_puppies', y='score', kind='scatter', color='b', label='puppy');

### Our model assumes that the slopes of these lines are the same

In [ ]:
scatter(p_data.num_puppies,
        results.fittedvalues[data.puppy == 1],
        color='m',
        marker='x',
        label='fitted puppies')
legend(loc=4);

In [ ]:
p_data.plot(x='num_puppies', y='score', kind='scatter', color='b', label='puppy', alpha=.2)
scatter(p_data.num_puppies,
        results.fittedvalues[data.puppy == 1],
        color='m',
        marker='x',
        label='fitted puppies')
legend(loc=4);

In [ ]:
lmplot('num_puppies', 'score', p_data, 'has_other_animals', fit_reg=False);

## Interactions

We can add interaction terms by multiplying columns together
$$y \sim \beta_0 + \beta_1 \times x_1 + \beta_2 \times x_2 + \beta_3 \times x_1 \times x_2 $$
so if $x_2$ is other animals:
$$y_i = \left\{\begin{array}{11} (\beta_0 + \beta_2) + (\beta_1 + \beta_3) \times x_{1i} + \epsilon_i & \mbox{if the ith photo has other animals} \\ \beta_0 + \beta_1 \times x_{1i} + \epsilon_i & \mbox{if ith picture has no other animals} \end{array} \right.$$

In [ ]:
lmplot('num_puppies', 'score', fake_puppy_data, 'has_other_animals', 'age_group');

To make our lives easier, we are going to start using R-style formulas courtesy of the library [`patsy`](http://patsy.readthedocs.io/en/latest/) which should have been installed automatically along with statsmodels.

In [ ]:
import statsmodels.formula.api as smf

In [ ]:
model2 = smf.ols(formula='score ~ num_puppies + has_other_animals + age_group', data=fake_puppy_data)
results2 = model2.fit()
results2.summary()

Notice that when we use a formula, `patsy` automatically adds an Intercept term and transforms categorical variables into dummies.

In [ ]:
data.head()

We can also use formulas to model interactions:

In [ ]:
formula = 'score ~ num_puppies * others_yes + puppy + elderly'

In [ ]:
model3 = smf.ols(formula = formula,data = data)
results3 = model3.fit()
results3.summary()

In [ ]:
p_data.plot(x='num_puppies',y='score',kind='scatter',color='b',label='puppy')
scatter(p_data.num_puppies, results3.fittedvalues[data['puppy']==1], color='y', marker='x' ,label = 'fitted puppies');

## Interactions between multiple continuous variables

We can add interaction terms by multiplying columns together
$$y \sim \beta_0 + \beta_1 \times x_1 + \beta_2 \times x_2 + \beta_3 \times x_1 \times x_2 $$
so if $x_1$ and $x_2$ are both continuous:
$$y_i = \beta_0 + (\beta_1 + \beta_3 \times x_{2i}) \times x_{1i} + \beta_2 \times x_{2i}$$

So, the value of the coefficient for $x_1$ now depends on the value of $x_2$.  
For interpretation you still consider the other variables held constant, but when there are interactions you cannot say how much one will improve without specifying the value the other one is held constant at.

## Stepwise Regression

Stepwise regression is a good simple tool for determining parameters to include in models. Care must be taken in its use, as it is easy to fall into the trap of trusting it more than you should. If you have a large number of parameters, or a small amount of data, it is probably best to use another tool. There are multiple ways to perform stepwise regression, the general procedures for stepwise regression are:

* Forward Stepwise Regression:
    * Start with some form of minimal model (like intercept only)
    * Add other variables into your model based on which ones maximally increase adjusted $R^2$ (or some other test criteria)
    * Continue until you run out of variables or adding any variable reduces the quality of your  model
* Backward Stepwise Regression:
    * Start with some form of maximal model (include all variables)
    * Remove variables that least improve the model (or have insignificant p-values), one at a time
    * Stop when you can no longer improve your model by removing variables
* Bidirectional Stepwise Regression:
    * Do both of the above

## Useful things to know

In order to implement stepwise regression you have to know how to compare models. There are several ways to do this:

* Adjusted $R^2$, AIC, BIC, F-test. These are model quality metrics (will discuss more on next slide)
* p-values for coefficients. These can be used to evaluate importance of individual predictors for the model.
* Others I haven't mentioned

## Model quality metrics
* Adjusted $R^2$  
$$Adjusted\ R^2 = 1 - {\frac{RSS}{(n-k-1)} \over \frac{TSS}{(n-1)}}$$
    * Want adjusted $R^2$ close to one, includes penalties for number of predictors (k)  

* AIC (Akaike Information Criterion)  
$$AIC = -2log(L) + 2k$$
    * L is likelihood for the model and k is number of predictors, want the model with the lowest AIC value  
    
* BIC (Bayesian Information Criterion)
$$BIC = -2log(L) + 2log(n)$$
    * Very similar to AIC, but since log(n) > 2 for n > 7 adds a larger penalty for additional predictors, once again want the lowest. n is the sample size.  

## Backward stepwise example

In [ ]:
boring_fake_data_1 = pd.DataFrame()
boring_fake_data_1['A'] = np.random.normal(25,5,1000)
boring_fake_data_1['C'] = np.random.normal(75,15,1000)
boring_fake_data_1['B'] = 5 * boring_fake_data_1['A'] - 2 * boring_fake_data_1['C'] + np.random.normal(20,3,1000)
boring_fake_data_1['D'] = np.random.uniform(-50,50,1000)
boring_fake_data_1['response'] = 50 * boring_fake_data_1['A'] + \
                                 3 * boring_fake_data_1['B'] + \
                                 50 * boring_fake_data_1['C'] + \
                                 50 + np.random.normal(350,150,1000)

In [ ]:
boring_fake_data_1.head()

## Fit the full model with all of the independent variables

In [ ]:
formula = 'response ~ A + B + C + D'
model1 = smf.ols(formula=formula, data=boring_fake_data_1)
results1 = model1.fit()
results1.summary()

## Eliminate the least significant variable

In [ ]:
formula = 'response ~ A + C + D'
model2 = smf.ols(formula=formula,data=boring_fake_data_1)
results2 = model2.fit()
results2.summary()

## And again

In [ ]:
formula = 'response ~ A + C'
model3 = smf.ols(formula=formula,data=boring_fake_data_1)
results3 = model3.fit()
results3.summary()